In [1]:
import pandas as pd
import numpy as np
import regex as re

In [2]:
df_all = pd.read_csv('rzepa_all.csv', index_col='id')
proper_cols = np.array(['title', 'magazine_nr', 'text', 'link', 'source', 'author','department', 'imgage', 'image_text', 'date','class'])

### Tagged data

In [7]:
data_1 = pd.read_csv("tagged/tagged_0.csv", index_col='id')
data_2 = pd.concat([pd.read_csv(f"tagged/updated_data_{i}.csv", index_col='id') for i in range(1,3)])
cols = ['magazine_nr','class', 'link'] 
data_list = [data[cols] for data in [data_1, data_2]]
df_12 = pd.concat(data_list)
df_12 = pd.merge(df_12, df_all, how='inner', on=['id','link', 'magazine_nr'])

In [9]:
data_3 = pd.read_csv("tagged/validation_0.csv", index_col='id')
data_3 = data_3[~data_3['rank_revised 1-5'].isna()]
data_3.loc[data_3['rank_revised 1-5'].apply(lambda x: True if re.search('5', x) else False), 'label'] = 0
data_3.rename(columns={'label': 'class'}, inplace=True)
data_3 = data_3[['magazine_nr', 'link', 'class']]
df_3 = pd.merge(data_3, df_all.reset_index(), how='inner', on = ['magazine_nr', 'link'])
df_3 = df_3.set_index('id')
df_3['magazine_nr'] = df_3['magazine_nr'].astype(int)

In [19]:
df_tagged_all = pd.concat([df_12, df_3])
print(len(df_tagged_all))
df_tagged_all = df_tagged_all.iloc[~df_tagged_all.index.duplicated(keep='first')]
print(len(df_tagged_all))

1379
1373


In [20]:
len(df_tagged_all), df_tagged_all.columns.values

(1373,
 array(['magazine_nr', 'class', 'link', 'title', 'text', 'source',
        'author', 'department', 'imgage', 'image_text', 'date'],
       dtype=object))

In [21]:
df_tagged_all['class'] = df_tagged_all['class'].astype(int)

In [22]:
one_to_zero = [28546, 92891, 16564,76958,29067,54264,164028, 25883,26107,164028,154940,40340,29306,59452,59560,213961]
one_to_zero.extend([20725,9445,157392,91486,191713,68595,92234,59598,29295,5495,29545,33137,214061,28996,214061,156058])
one_to_zero.extend([29311,31709,27281,52829,93483,59079,144102,29311,30376,27773,192071,160022,91673,60016,29827,22455,4091,93485,59301,25138,59997,213409,213771,2906,150789,122132,122293,24594,130858,213120,92735,192071])
one_to_zero.extend([192012, 118761,  27281,  92735, 122137,  59997, 122717,  30376,92109,  31709,  22455, 213120, 213983])
one_to_zero.extend([147521, 100383,  88415, 113014, 198686, 213536, 210098, 118933,
       128253, 198685, 120812, 123203, 106132, 150356, 159867, 155776,
       152903, 200337, 194524, 110438, 149109, 213246, 203029,
       133157, 213907, 122621, 160077, 178269, 160965, 189486, 102565,
       213842, 202292, 183495, 112524,  95627,  65447, 122011, 191277,
       136626, 213476,  99147, 111853, 187641, 157392, 194987,
       211263, 187037, 149484, 103954, 186329, 202952, 127724, 115554])
one_to_zero = [*set(one_to_zero)]
print(len(one_to_zero))
zero_to_one = [177788, 191713, 157392, 116963]
zero_to_one.extend([160022, 154959,  93485,  91486, 192071,  56463, 162557,   2906,
        36234,  33565, 118761, 144102,  25867,  41358,  29827,   9445,
        21000,  35271, 214061,  33137,  30376, 213771,  23863])

120


In [23]:
df_tagged_all.loc[one_to_zero, 'class'] = np.zeros_like(one_to_zero)
df_tagged_all.loc[zero_to_one, 'class'] = np.ones_like(zero_to_one)
df_tagged_all['translated'] = np.zeros_like(df_tagged_all.index)
df_tagged_all.to_csv('dataset/dataset_6.csv')

KeyError: '[93350] not in index'

In [ ]:
data_translated = pd.concat([pd.read_csv(f'tagged/translated/translated_{i}.csv', index_col='id') for i in range(2)])
translated_1_to_0 = data_translated[data_translated.index.isin(one_to_zero)].index.values
translated_0_to_1 = data_translated[data_translated.index.isin(zero_to_one)].index.values

In [ ]:
def author_extract(text):
    extracted = re.search('((?<=\\t—).*)|(.*(?=@rp\.pl))', text)
    return extracted[0] if extracted is not None else extracted

def restore_authors(df):
    df['last'] = df['text'].apply(lambda x: x.split(' ')[-1])
    df['extracted'] = df['last'].apply(author_extract)
    df.loc[df['author']=='None', 'author'] = df["extracted"]
    df.drop(columns = ['last', 'extracted'], inplace = True)
    return df

In [ ]:
data_translated.loc[translated_1_to_0, 'class'] = np.zeros_like(translated_1_to_0) # these misclassified samples were present
data_translated.loc[translated_0_to_1, 'class'] = np.ones_like(translated_0_to_1) # these misclassified samples were present
data_translated = data_translated[~data_translated.index.duplicated(keep='first')]
data_translated['translated'] = np.ones_like(data_translated.index)
data_translated['class'] = data_translated['class'].astype(int)
data_translated = restore_authors(data_translated)

In [ ]:
data_translated.to_csv('dataset/dataset_6_translated.csv')

In [ ]:
!rm 'rzepa_all.csv'